In [1]:
import requests as r
import re
import os
import logging
from datetime import datetime, timedelta
from dotenv import load_dotenv
import logging
from shapely import from_wkt
from shapely.geometry import Polygon, MultiPolygon

In [2]:
load_dotenv()

True

In [3]:
def get_access_token(client_id: str, client_secret: str) -> str:
    '''
    Supplies a temporary access token for of the OS NGD API
    Times out after 5 minutes
    Takes the project client_id and client_secret as input
    '''

    url = "https://api.os.uk/oauth2/token/v1"

    data = {
        "grant_type": "client_credentials"
    }

    response = r.post(
        url, 
        auth=(client_id, client_secret),
        data=data
    )

    json_response = response.json()
    if response.status_code >= 400:
        raise Exception(json_response)
    token = json_response["access_token"]

    return token

In [4]:
def OAauth2_manager(func: callable):

    def wrapper(*args, **kwargs):

        kwargs_ = kwargs.copy()

        try:
            access_token = os.environ.get('ACCESS_TOKEN')
            kwargs_['access_token'] = access_token
            return func(*args, **kwargs_)
        except Exception:
            client_id = os.environ.get('CLIENT_ID')
            client_secret = os.environ.get('CLIENT_SECRET')
            access_token = get_access_token(
                client_id=client_id,
                client_secret=client_secret
            )
            os.environ['ACCESS_TOKEN'] = access_token
            kwargs_['access_token'] = access_token
            return func(*args, **kwargs_)

    wrapper.__name__ = func.__name__ + '+OAuth2_manager'
    funcname = func.__name__
    wrapper.__doc__ = f"""
    An extension of the function {funcname} handling OAauth2 authorisation.
    IMPORTANT:
        CLIENT_ID and CLIENT_SECRET must be set as environment variables for this extension to work.
        This can be done using a .env file and load_dotenv()
    Docs for OAuth2 with Ordnance Survey data can be found at https://osdatahub.os.uk/docs/oauth2/overview
    The function automatically ensures a valid temporary access token (expiring after 5 minutes) is being used. This will either be an existing valid token, or a newly called one.

    ____________________________________________________
    Docs for {funcname}:
        {func.__doc__}
    """
    return wrapper

In [5]:
def generic_request(url: str, headers: dict = {}, access_token: str = None, **kwargs) -> dict:

    if access_token:
        headers['Authorization'] = f"Bearer {access_token}"
    response = r.get(url, headers=headers, **kwargs)
    json_response = response.json()
    if response.status_code >= 400:
        raise Exception(json_response)

    return json_response

In [6]:
def construct_query_params(**kwargs) -> str:
    '''
    Constructs a query string from a dictionary of key-value pairs.
    Refer to https://osdatahub.os.uk/docs/ofa/technicalSpecification for details about query parameters.
    The options are:
        - bbox
        - bbox-crs
        - crs
        - datetime
        - filter
        - filter-crs
        - filter-lang
        - limit
        - offset
    '''
    params_list = [f'{k}={v}' for k, v in kwargs.items()]
    return '?' + '&'.join(params_list)

In [7]:
def wkt_to_spatial_filter(wkt):
    return f'(INTERSECTS(geometry,{wkt}))'

In [8]:
def construct_filter_param(**kwargs):
    for k, v in kwargs.items():
        if type(v) == str:
            kwargs[k] = f"'{v}'"
    filter_list = [f"({k}={v})" for k, v in kwargs.items()]
    return 'and'.join(filter_list)

In [9]:
def ngd_items_request(
    collection: str,
    query_params: dict = {},
    filter_params: dict = {},
    wkt = None,
    headers: dict = {},
    access_token: str = None,
    **kwargs
) -> dict:
    """
    Calls items from the OS NGD API - Features
        - https://osdatahub.os.uk/docs/wfs/overview
        - https://docs.os.uk/osngd/accessing-os-ngd/access-the-os-ngd-api/os-ngd-api-features
    Parameters:
        collection (str) - the feature collection to call from. Feature collection names and details can be found at https://api.os.uk/features/ngd/ofa/v1/collections/
        query_params (dict, optional) - parameters to pass to the query as query parameters, supplied in a dictionary. Supported parameters are: bbox, bbox-crs, crs, datetime, filter, filter-crs, filter-lang, limit, offset
        filter_params (dict, optional) - OS NGD attribute filters to pass to the query within the 'filter' query_param. The can be used instead of or in addition to manually setting the filter in query_params.
            The key-value pairs will appended using the EQUAL TO [ = ] comparator. Any other CQL Operator comparisons must be set manually in query_params.
            Queryable attributes can be found in OS NGD codelists documentation https://docs.os.uk/osngd/code-lists/code-lists-overview, or by inserting the relevant collectionId into the https://api.os.uk/features/ngd/ofa/v1/collections/{{collectionId}}/queryables endpoint.
        wkt (string or shapely geometry object) - A means of searching a geometry for features. The search area(s) must be supplied in wkt, either in a string or as a Shapely geometry object.
            The function automatically composes the full INTERSECTS filter and adds it to the 'filter' query parameter.
            Make sure that 'filter-crs' is set to the appropriate value.
        headers (dict, optional) - Headers to pass to the query. These can include bearer-token authentication.
        access_token (str) - An access token, which will be added as bearer token to the headers.
        **kwargs: other generic parameters to be passed to the requests.get()

    Returns the features as a geojson, as per the OS NGD API.
    """

    query_params_ = query_params.copy()
    filter_params_ = filter_params.copy()
    headers_ = headers.copy()

    if filter_params_:
        filters = construct_filter_param(**filter_params_)
        current_filters = query_params_.get('filter')
        query_params_['filter'] = f'({current_filters})and{filters}' if current_filters else filters

    if wkt:
        spatial_filter = wkt_to_spatial_filter(wkt)
        current_filters = query_params_.get('filter')
        query_params_['filter'] = f'({current_filters})and{spatial_filter}' if current_filters else spatial_filter


    query_params_string = construct_query_params(**query_params)
    url = f'https://api.os.uk/features/ngd/ofa/v1/collections/{collection}/items/{query_params_string}'
    if access_token:
        headers_['Authorization'] = f"Bearer {access_token}"
    response = r.get(url, headers=headers_, **kwargs)
    json_response = response.json()

    if response.status_code >= 400:
        raise Exception(json_response)

    return json_response

In [10]:
def get_latest_collection_versions(flag_recent_updates: bool = True, recent_update_days: int = 31) -> tuple[dict[str: str], list[str]]:
    '''
    Returns the latest collection versions of each NGD collection.
    Feature collections follow the following naming convention: theme-collection-featuretype-version (eg. bld-fts-buildingline-2)
    The output of this function maps base feature collection names (theme-collection-featuretype) to the full name, including the latest version.
    This can be used to ensure that software is always using the latest version of a feature collection.
    More details on feature collection naming can be found at https://docs.os.uk/osngd/accessing-os-ngd/access-the-os-ngd-api/os-ngd-api-features/what-data-is-available
    '''

    response = r.get('https://api.os.uk/features/ngd/ofa/v1/collections/')
    collections_data = response.json()['collections']
    collections_list = [collection['id'] for collection in collections_data]
    collection_base_names = set([re.sub(r'-\d+$', '', c) for c in collections_list])
    output_lookup = dict()

    for base_name in collection_base_names:
        all_versions = [c for c in collections_list if c.startswith(base_name)]
        latest_version = max(all_versions, key=lambda c: int(c.split('-')[-1]))
        output_lookup[base_name] = latest_version

    recent_collections = None
    if flag_recent_updates:
        time_format = r'%Y-%m-%dT%H:%M:%SZ'
        recent_update_cutoff = datetime.now() - timedelta(days=recent_update_days)
        latest_versions_data = [c for c in collections_data if c['id'] in output_lookup.values()]
        recent_collections = list()
        for collection_data in latest_versions_data:
            version_startdate = collection_data['extent']['temporal']['interval'][0][0]
            time_obj = datetime.strptime(version_startdate, time_format)
            if time_obj > recent_update_cutoff:
                collection = collection_data['id']
                recent_collections.append(collection)
                logging.warning(f'{collection} is a recent version/update from the last {recent_update_days} days.')

    return output_lookup, recent_collections

In [11]:
def get_single_latest_collection(collection: str, **kwargs) -> str:
    '''
    Returns the latest collection of a given collection base.
    Input must be in the format theme-collection-featuretype (eg. bld-fts-buildingline)
    Output will complete the full name of the feature collection by appending the latest version number (eg. bld-fts-buildingline-2)
    More details on feature collection naming can be found at https://docs.os.uk/osngd/accessing-os-ngd/access-the-os-ngd-api/os-ngd-api-features/what-data-is-available
    '''
    latest_collections = get_latest_collection_versions(**kwargs)
    latest_collection = latest_collections[collection]
    return latest_collection

In [12]:
def multiple_collections_extension(func: callable) -> dict:

    def wrapper(collections: list[str], *args, **kwargs):

        results = dict()
        for c in collections:
            json_response = func(c, *args, **kwargs)
            results[c] = json_response
        return results
    
    wrapper.__name__ = func.__name__ + '+multiple_collections_extension'
    funcname = func.__name__
    wrapper.__doc__ = f"""
    Extents the {funcname} function to handle multiple collections.
    Takes a list of collection names as input, alongside any other parameters which are passed to {funcname}.
    The function {funcname} will be run for each collection in turn, with the results returned in a dictionary mapping the collection names to the results.
    NOTE: If a limit is supplied for the maximum number of features to be returned or requests to be made, this will apply to each collection individually, not to the overall number of results.

    ____________________________________________________
    Docs for {funcname}:
        {func.__doc__}
    """
    return wrapper

In [13]:
def feature_limit_extension(func: callable):

    def wrapper(
        *args,
        request_limit: int = 50,
        feature_limit: int = None,
        query_params: dict = {},
        **kwargs
    ):

        query_params_ = query_params.copy()

        if 'offset' in query_params_:
            raise AttributeError('offset is not a valid argument for functions using this decorator.')

        items = list()

        batch_count, final_batchsize = divmod(feature_limit, 100) if feature_limit else (None, None)
        request_count = 0
        offset = 0

        if not(feature_limit) and not(request_limit):
            raise AttributeError('At least one of feature_limit or request_limit must be provided to prevent indefinitely numerous requests and high costs. However, there is no upper limit to these values.')

        while (request_count != request_limit) and (not(feature_limit) or offset < feature_limit):

            if request_count == batch_count:
                print('final batch of size', final_batchsize)
                query_params_['limit'] = final_batchsize
            query_params_['offset'] = offset

            json_response = func(*args, query_params=query_params_, **kwargs)
            items += json_response['features']

            if not [link for link in json_response['links'] if link['rel'] == 'next']:
                break

            request_count += 1
            offset += 100

        geojson = {
            "type": "FeatureCollection",
            "numberOfRequests": request_count + 1,
            "totalNumberReturned": len(items),
            "timeStamp": datetime.now().isoformat(),
            "collection": kwargs.get('collection'),
            "source": "Compiled from code by Geovation from Ordnance Survey",
            "features": items
        }
        return geojson

    wrapper.__name__ = func.__name__ + '+feature_limit_extension'
    funcname = func.__name__
    wrapper.__doc__ = f"""
    This is an extension the {funcname} function, which returns OS NGD features. It serves to extend the maximum number of features returned above the default maximum 100 by looping through multiple requests.
    It takes the following arguments:
    - collection: The name of the collection to be queried.
    - request_limit: The maximum number of calls to be made to {funcname}. Default is 50.
    - feature_limit: The maximum number of features to be returned. Default is None.
    - query_params: A dictionary of query parameters to be passed to the function. Default is an empty dictionary.
    To prevent indefinite requests and high costs, at least one of feature_limit or request_limit must be provided, although there is no limit to the upper value these can be.
    It will make multiple requests to the function to compile all features from the specified collection, returning a dictionary with the features and metadata.

    ____________________________________________________
    Docs for {funcname}:
        {func.__doc__}
    """
    return wrapper

In [14]:
def multigeometry_search_extension(func: callable):

    def wrapper(*args, wkt: str, **kwargs):

        multi_geom = from_wkt(wkt) if type(wkt) == str else wkt
        search_areas = list()

        for search_area, geom in enumerate(multi_geom.geoms):
            json_response = func(*args, wkt=geom, **kwargs)
            json_response['searchAreaNumber'] = search_area
            search_areas.append(json_response)

        geojson = {
            "type": "FeatureCollection",
            "searchAreas": search_areas
        }

        return geojson

    wrapper.__name__ = func.__name__ + '+multigeometry_search_extension'
    funcname = func.__name__
    wrapper.__doc__ = f"""
    An alternative means of returning OS NGD features for a search area which is a Multi-Geometry (MultiPoint, MultiLinestring, or MultiPolygon), which will in some cases improve speed, performance, and prevent the call from timing out.
    Extends to {funcname} function.
    Each component shape of the multi-geometry will be searched in turn using the {funcname} function.
    The results are returned in a quasi-GeoJSON format, with features returned under 'searchAreas' in a list, where each item is a json object of results from one search area.
    The search areas are labelled numerically, with the number stored under 'searchAreaNumber'.
    NOTE: If a limit is supplied for the maximum number of features to be returned or requests to be made, this will apply to each search area individually, not to the overall number of results.

    ____________________________________________________
    Docs for {funcname}:
        {func.__doc__}
    """
    return wrapper

In [15]:
x=Polygon({(400000,400000), (400050,400000), (400050,400050), (400000,40050)})
y=Polygon({(500000,500000), (500050,500000), (500050,500050), (500000,50050)})

In [16]:
z=MultiPolygon([x,y])

# Example Ways of Building Custom APIs by Combining Wrappers

#### All possible combos with OAuth2

In [25]:
items_auth = OAauth2_manager(ngd_items_request)
items_auth_limit = feature_limit_extension(items_auth)
items_auth_limit_geom = multigeometry_search_extension(items_auth_limit)
items_auth_limit_geom_col = multiple_collections_extension(items_auth_limit_geom)

items_auth_geom = multigeometry_search_extension(items_auth)
items_auth_geom_col = multiple_collections_extension(items_auth_geom)

items_auth_limit_col = multiple_collections_extension(items_auth_limit)

items_auth_col = multiple_collections_extension(items_auth)


## Demos

In [ ]:
everything_test = items_auth_limit_geom_col(
    collections=['lnd-fts-land-3','wtr-fts-water-1','bld-fts-building-3'],
    wkt = z,
    query_params={'filter-crs':'http://www.opengis.net/def/crs/EPSG/0/27700'},
    request_limit=10
)